In [136]:
import pandas as pd
import numpy as np
import math
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold

import platform
from os import listdir
from os.path import isfile, join
from glob import glob
from pathlib import Path
import sys
import os
import copy
import traceback
from pathlib import Path


import matplotlib.pyplot as plt

import SMOTE
import feature_selector
import DE
import CFS
import birch
import metrics.abcd

import metrices
import measures

import sys
import traceback
import warnings
warnings.filterwarnings("ignore")

In [139]:
data_source1 = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/src/data/1385/exp1/2'
if platform.system() == 'Darwin' or platform.system() == 'Linux':
    _dir = data_source1 + '/'
else:
    _dir = data_source1 + '\\'

clusters = [(join(_dir, f)) for f in listdir(_dir) if Path(join(_dir, f)).is_dir()]

In [174]:
count = 0
count_not = 0
count_yes = 0
precentage = 0
score = []
for cluster in clusters:
    df = pd.read_csv(cluster + '/1385_LR_bellwether_g.csv')
    counts = []
    for row in range(df.shape[0]):
        j = df.iloc[row].values[1:]
        value = sum(i >= 0.66 for i in j)
        counts.append(value)
    df['>=0.6'] = counts
    precentage = max(counts)/df.shape[0]
    score.append([int(cluster.rsplit('/',1)[1]),round(df.shape[0]*0.2),max(counts),round(np.mean(counts),2)])
    count += max(counts)

In [175]:
score_df = pd.DataFrame(score, columns = ['id','expected','max_bell','avge'])

In [176]:
score_df = score_df.sort_values('id')

In [177]:
score_df.to_csv('data/1385/projects/bellwether.csv')

In [165]:
count

617